In [1]:
import torch
import pyro
import pyro.distributions as dist
import pyro.distributions.transforms as T
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

data1 = [1 if random.random() < 0.3 else 0 for i in range(10000)]
data2 = [1 if random.random() < 0.6 else 0 for i in range(10000)]
data = torch.stack([torch.tensor(data1), torch.tensor(data2)], dim=-1).float()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
data = data.to(device)

In [10]:
base_dist = dist.Normal(torch.zeros(2).to(device), torch.ones(2).to(device))
spline1 = T.spline(2).to(device)
spline2 = T.spline(2).to(device)
flow_dist = dist.TransformedDistribution(base_dist, [spline1, spline2])

In [11]:
optimizer = torch.optim.Adam(list(spline1.parameters()) + list(spline2.parameters()), lr=5e-2)
for step in range(1, 502):
    optimizer.zero_grad()
    b = pyro.distributions.Bernoulli(flow_dist.rsample(torch.Size([10000,])).sigmoid().mean(dim=0))
    loss = -b.log_prob(data).mean()
    loss.backward()
    optimizer.step()
    flow_dist.clear_cache()

    if step % 50 == 1:
        print('step: {}, loss: {}'.format(step, loss.item()))

step: 1, loss: 0.6591983437538147
step: 51, loss: 0.6430516242980957
step: 101, loss: 0.6430274248123169
step: 151, loss: 0.6430831551551819
step: 201, loss: 0.643086314201355
step: 251, loss: 0.6430315375328064
step: 301, loss: 0.6430330872535706
step: 351, loss: 0.643073320388794
step: 401, loss: 0.6430783271789551
step: 451, loss: 0.6430176496505737
step: 501, loss: 0.6430212259292603


In [12]:
b = pyro.distributions.Bernoulli(flow_dist.rsample(torch.Size([10000,])).sigmoid().mean(dim=0))

In [13]:
b.mean

tensor([0.3033, 0.5992], device='cuda:0', grad_fn=<MeanBackward1>)

In [14]:
data.mean(dim=0)

tensor([0.3043, 0.6035], device='cuda:0')